
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [1034]:
# (this is not an exhaustive list of libraries)
import pandas as pd
from pandas import json_normalize
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags
from sklearn.preprocessing import LabelEncoder

In [1035]:
# load one file first to see what type of data you're dealing with and what attributes it has

path = r"C:\Users\turab\data-project-midterm\data"

# Use os.listdir to get files
files = os.listdir(directory)

# Print all files
for file in files:
    print(file)

.gitkeep
AK_Juneau_0.json
AK_Juneau_1.json
AK_Juneau_2.json
AK_Juneau_3.json
AK_Juneau_4.json
AL_Montgomery_0.json
AL_Montgomery_1.json
AL_Montgomery_2.json
AL_Montgomery_3.json
AL_Montgomery_4.json
AR_LittleRock_0.json
AR_LittleRock_1.json
AR_LittleRock_2.json
AR_LittleRock_3.json
AR_LittleRock_4.json
AZ_Phoenix_0.json
AZ_Phoenix_1.json
AZ_Phoenix_2.json
AZ_Phoenix_3.json
AZ_Phoenix_4.json
CA_Sacramento_0.json
CA_Sacramento_1.json
CA_Sacramento_2.json
CA_Sacramento_3.json
CA_Sacramento_4.json
CO_Denver_0.json
CO_Denver_1.json
CO_Denver_2.json
CO_Denver_3.json
CO_Denver_4.json
CT_Hartford_0.json
CT_Hartford_1.json
CT_Hartford_2.json
CT_Hartford_3.json
CT_Hartford_4.json
DE_Dover_0.json
DE_Dover_1.json
DE_Dover_2.json
DE_Dover_3.json
DE_Dover_4.json
FL_Tallahassee_0.json
FL_Tallahassee_1.json
FL_Tallahassee_2.json
FL_Tallahassee_3.json
FL_Tallahassee_4.json
GA_Atlanta_0.json
GA_Atlanta_1.json
GA_Atlanta_2.json
GA_Atlanta_3.json
GA_Atlanta_4.json
HI_Honolulu_0.json
HI_Honolulu_1.json
HI_

Combine all sales into dataframe

In [1072]:
def load_and_combine_json_files(directory):
    data_frames = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                
                # Normalize the JSON structure to a flat table
                if 'data' in json_data and 'results' in json_data['data']:
                    df = json_normalize(json_data['data']['results'])
                    data_frames.append(df)
    
    # Concatenate all DataFrames into one
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

directory = r"C:\Users\turab\data-project-midterm\data"  # Update this path

# Load and combine the JSON files
combined_data = load_and_combine_json_files(directory)

print(combined_data.head())

C:\Users\turab\AppData\Local\Temp\ipykernel_5512\2437266733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(data_frames, ignore_index=True)


       last_update_date                                               tags  \
0  2023-09-19T20:52:50Z  [carport, community_outdoor_space, cul_de_sac,...   
1                  None                                               None   
2                  None                                               None   
3                  None                                               None   
4                  None                                               None   

   ... community.description.name location.county  
0  ...                        NaN             NaN  
1  ...                        NaN             NaN  
2  ...                        NaN             NaN  
3  ...                        NaN             NaN  
4  ...                        NaN             NaN  

[5 rows x 67 columns]


At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

Drop or replace values as neccesary 



In [1090]:
# Check if some columns have lists

list_columns_combined_data = check_for_lists(combined_data)
print(list_columns_combined_data)

# Tags, branding, photos, virtual tours, source agents, other listings, etc have lists!

{'last_update_date': False, 'tags': True, 'permalink': False, 'status': False, 'list_date': False, 'open_houses': False, 'branding': True, 'list_price': False, 'property_id': False, 'photos': True, 'community': False, 'virtual_tours': True, 'listing_id': False, 'price_reduced_amount': False, 'matterport': False, 'primary_photo.href': False, 'source.plan_id': False, 'source.agents': True, 'source.spec_id': False, 'source.type': False, 'description.year_built': False, 'description.baths_3qtr': False, 'description.sold_date': False, 'description.sold_price': False, 'description.baths_full': False, 'description.name': False, 'description.baths_half': False, 'description.lot_sqft': False, 'description.sqft': False, 'description.baths': False, 'description.sub_type': False, 'description.baths_1qtr': False, 'description.garage': False, 'description.stories': False, 'description.beds': False, 'description.type': False, 'lead_attributes.show_contact_an_agent': False, 'flags.is_new_construction'

In [1091]:
# Display all from tags column 

if 'tags' in combined_data.columns:
    # Initialize an empty set to store unique tags
    unique_tags = set()
    
    # Iterate over each entry in the 'tags' column
    for tag_list in combined_data['tags']:
        if isinstance(tag_list, list):  # Ensure the entry is a list
            unique_tags.update(tag_list)  # Add tags to the set, avoiding duplicates
    
    # Print the unique set of tags
    print("Unique tags:")
    for tag in unique_tags:
        print(tag)
else:
    print("'tags' column does not exist in the DataFrame.")

Unique tags:
clubhouse
community_clubhouse
jack_and_jill_bathroom
basketball_court
wrap_around_porch
courtyard_entry
library
fenced_yard
carport
lake
garage_2_or_more
lake_view
trails
baseball
garage_3_or_more
community_elevator
tennis
boat_dock
pond
single_story
park
laundry_room
hill_or_mountain_view
garage_1_or_more
community_center
community_park
forced_air
wine_cellar
hardwood_floors
two_or_more_stories
golf_course_lot_or_frontage
fixer_upper
first_floor_master_bedroom
indoor_basketball_court
high_ceiling
master_bathroom
central_air
hoa
outdoor_kitchen
mountain_view
views
pets_allowed
community_horse_facilities
large_porch
city_view
ranch
marina
community_spa_or_hot_tub
community_outdoor_space
family_room
water_view
private_backyard
beach
disability_features
ensuite
ocean_view
smart_homes
community_swimming_pool
outbuilding
efficient
community_tennis_court
central_heat
fruit_trees
guest_parking
white_kitchen
furniture
den_or_office
open_house
master_bedroom
guest_house
updated_kit

In [1092]:
# Data types of each column

data_types_list = combined_data.dtypes.tolist()

print("Data types of each column:")
print(data_types_list)

# There are objects, float64's

Data types of each column:
[dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64')]


In [1093]:
# drop sales that dont include sales price

combined_data = combined_data.dropna(subset=['description.sold_price'])

# Print the shape of the DataFrame to verify the number of rows after dropping
print("Shape of DataFrame after dropping rows with missing sales price:", combined_data.shape)

Shape of DataFrame after dropping rows with missing sales price: (6716, 67)


In [1094]:
# Drop sales that dont have a property type
# Display the property type column

df_filtered = combined_data.dropna(subset=['description.sub_type'])
df_filtered = df_filtered[df_filtered['description.sub_type'].str.lower() != 'none']

# Print the shape of the DataFrame to verify the number of rows after dropping
print("Shape of DataFrame after dropping rows with missing property type and 'None':", df_filtered.shape)


Shape of DataFrame after dropping rows with missing property type and 'None': (1172, 67)


Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [1095]:
# See what categorical columns exist including lists or dicts

categorical_columns = df_filtered.select_dtypes(include=['object']).columns
print(categorical_columns)

Index(['last_update_date', 'tags', 'permalink', 'status', 'list_date',
       'open_houses', 'branding', 'property_id', 'photos', 'community',
       'virtual_tours', 'listing_id', 'matterport', 'primary_photo.href',
       'source.plan_id', 'source.agents', 'source.spec_id', 'source.type',
       'description.sold_date', 'description.name', 'description.sub_type',
       'description.baths_1qtr', 'description.type',
       'lead_attributes.show_contact_an_agent', 'flags.is_new_construction',
       'flags.is_for_rent', 'flags.is_subdivision', 'flags.is_contingent',
       'flags.is_price_reduced', 'flags.is_pending', 'flags.is_foreclosure',
       'flags.is_plan', 'flags.is_coming_soon', 'flags.is_new_listing',
       'products.brand_name', 'other_listings.rdc',
       'location.address.postal_code', 'location.address.state',
       'location.address.city', 'location.address.state_code',
       'location.address.line', 'location.street_view_url',
       'location.county.fips_code', 'l

In [1096]:
# Remove photos column

df_filtered = df_filtered.drop(columns=['photos'])


In [1097]:
# Flatten the lists in the 'tags' column
df_filtered['tags'] = df_filtered['tags'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# Perform one-hot encoding on the flattened 'tags' column

df_filtered = pd.get_dummies(df_filtered, columns=['tags'])

In [1098]:
# Initialize LabelEncoder

label_encoder = LabelEncoder()

# Encode the 'permalink' column

df_filtered['permalink_encoded'] = label_encoder.fit_transform(df_filtered['permalink'])

# Drop the original 'permalink' column

df_filtered.drop(columns=['permalink'], inplace=True)

In [1099]:
# Encode 'sold' column

status_onehot = pd.get_dummies(df_filtered['status'], prefix='status')

df_filtered = pd.concat([df_filtered, status_onehot], axis=1)

# Drop the original 'status' column
df_filtered.drop(columns=['status'], inplace=True)

In [1100]:
# Convert dictionaries to strings in the 'branding' column
df_filtered['branding'] = df_filtered['branding'].apply(lambda x: str(x))

# Perform one-hot encoding on the 'branding' column
one_hot_encoded_branding = pd.get_dummies(df_filtered['branding'], prefix='branding')

# Concatenate the one-hot encoded columns with the original DataFrame
df_filtered = pd.concat([df_filtered, one_hot_encoded_branding], axis=1)

# Drop the original 'branding' column
df_filtered.drop(columns=['branding'], inplace=True)



In [1101]:
# Replace negative placeholder values with None

df_filtered = df_filtered.replace(-9223372037, np.nan)


In [1103]:
# Function to flatten lists and dictionaries

def flatten_value(x):
    if isinstance(x, list):
        return ','.join(map(str, x))
    elif isinstance(x, dict):
        return ','.join(str(v) for v in x.values())
    else:
        return str(x) if x is not None else ''

# Flatten lists or dictionaries in all columns

for col in df_filtered.columns:
    df_filtered[col] = df_filtered[col].apply(flatten_value)

# Convert datetime strings to Unix timestamps (integers)

datetime_columns = ['last_update_date', 'list_date', 'description.sold_date']
for col in datetime_columns:
    df_filtered[col] = pd.to_datetime(df_filtered[col], errors='coerce').astype('int64') // 10**9  # Convert to Unix timestamp

# One-hot encode all columns, including datetime

encoded_df = pd.get_dummies(df_filtered, dummy_na=True)


In [1104]:
# Concatenate the original DataFrame with the encoded DataFrame along the columns axis

merged_df = pd.concat([df_filtered, encoded_df], axis=1)

# Check the merged dataframe

merged_df.head()

,last_update_date,list_date,...,"branding_[{'name': None, 'photo': None, 'type': 'Office'}]_True","branding_[{'name': None, 'photo': None, 'type': 'Office'}]_nan"
273,1704852385,1701483574,...,False,False
314,1704852385,1701483574,...,False,False
355,1704852385,1701483574,...,False,False
396,1704852385,1701483574,...,False,False
437,1704852385,1701483574,...,False,False


- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [1105]:
# perform train test split here

from sklearn.model_selection import train_test_split

X = merged_df.drop('description.sold_price', axis=1)  # dropping the 'list_price' column as mentioned
y = merged_df['description.sold_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# do something with state and city
# drop any other not needed columns

In [1106]:
# Save data to csvs

X_train.to_csv('x_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [132]:
# import, join and preprocess new data here

,col1_1,col1_2,"col1_3,4",col1_5,col2_a,col2_b,col2_c,col2_d,"col3_1,2","col3_3,4",col3_e,col3_f
0,True,False,False,False,True,False,False,False,True,False,False,False
1,False,True,False,False,False,True,False,False,False,False,True,False
2,False,False,True,False,False,False,True,False,False,False,False,True
3,False,False,False,True,False,False,False,True,False,True,False,False


Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized